In [64]:
from scipy.stats import chi2, norm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from scipy.optimize import minimize, least_squares

# a)

In [65]:
data = pd.read_csv('Nerlove1963.csv').apply(np.log)
y = data['Cost'].values
data.head()

,Cost,output,Plabor,Pcapital,Pfuel
0,-2.501036,0.693147,0.737164,5.209486,2.884801
1,-0.414001,1.098612,0.717840,5.159055,3.558201
2,-0.010050,1.386294,0.717840,5.141664,3.558201
3,-1.155183,1.386294,0.604316,5.111988,3.471966
4,-1.624552,1.609438,0.751416,5.451038,3.353407


In [66]:
dat = data.drop(['Cost'], axis=1)

x = dat.values
x = np.stack([x[:, 0],
              x[:, 1:].sum(axis=1),
              x[:, 0]], axis=1)
x = sm.add_constant(x)

n = x.shape[0]

names = ['b1', 'b2', 'b3', 'b4', 'gamma']

dat.head()

,output,Plabor,Pcapital,Pfuel
0,0.693147,0.737164,5.209486,2.884801
1,1.098612,0.717840,5.159055,3.558201
2,1.386294,0.717840,5.141664,3.558201
3,1.386294,0.604316,5.111988,3.471966
4,1.609438,0.751416,5.451038,3.353407


In [67]:
print("Range of logQ is: (",
      round(dat['output'].sort_values().values[10], 2),
      ", ",
      round(dat['output'].sort_values().values[-10], 2),
      ")", sep='')

Range of logQ is: (3.22, 8.97)


# b)

In [68]:
# NLLS approach
def get_y(x, b):
  y_part = x[:, :-1] @ b[:-2]
  return y_part + b[-2] * x[:, -1] / (1 + np.exp(b[-1] - x[:, -1]))

def get_residuals(b):
    return y - get_y(x, b)

def get_residuals_jac(b):
    return - np.concatenate([x[:, :-1],
                             x[:, -1:] / (1 + np.exp(b[-1] - x[:, -1:])),
                             - np.exp(b[-1] - x[:, -1:]) * b[-2] * x[:, -1:]
                             / (1 + np.exp(b[-1] - x[:, -1:])) ** 2
                             ],
                             axis=1)

def get_Vbeta(b):

  X_quasi = - get_residuals_jac(b)
  Qgg_hat = X_quasi.T @ X_quasi / n
  Qgg_hat_inv = np.linalg.inv(Qgg_hat)
  e_hat = get_residuals(b)
  Vge_hat = X_quasi.T @ (X_quasi * (e_hat ** 2)[:, np.newaxis]) / n
  Vbeta_hat = Qgg_hat_inv @ Vge_hat @ Qgg_hat_inv

  return Vbeta_hat

In [69]:
start = [[0, 0, 0, 0, 4], [1, 1, 1, 1, 7], [1, -1, 0, 1, 6]]

ar = []

for sample in start:

  result = least_squares(
      fun=get_residuals,
      x0=sample,
      jac=get_residuals_jac,)

  res = result.x
  ar.append(res)

  se = np.diag(get_Vbeta(res) / n) ** 0.5

  print("Start values are: ", sample)
  for name, b, s in zip(names, res, se):
    print(name, ": ", round(b, 2), " (", round(s, 2), ")" , sep='')
  print()

Start values are:  [0, 0, 0, 0, 4]
b1: -5.32 (0.49)
b2: 0.44 (0.1)
b3: 0.37 (0.04)
b4: 0.22 (0.05)
gamma: 6.87 (0.36)

Start values are:  [1, 1, 1, 1, 7]
b1: -5.32 (0.49)
b2: 0.44 (0.1)
b3: 0.37 (0.04)
b4: 0.22 (0.05)
gamma: 6.88 (0.36)

Start values are:  [1, -1, 0, 1, 6]
b1: -5.32 (0.49)
b2: 0.44 (0.1)
b3: 0.37 (0.04)
b4: 0.22 (0.05)
gamma: 6.87 (0.36)



Here we were lucky with initial values. However, if we used too small gamma, things could have got worse, and we would have arrived in another optimum:

In [70]:
start = [[-1, 0, 1, 2, 3.22]]

ar = []

for sample in start:

  result = least_squares(
      fun=get_residuals,
      x0=sample,
      jac=get_residuals_jac,)

  res = result.x
  ar.append(res)

  se = np.diag(get_Vbeta(res) / n) ** 0.5

  print("Start values are: ", sample)
  for name, b, s in zip(names, res, se):
    print(name, ": ", round(b, 2), " (", round(s, 2), ")" , sep='')
  print()

Start values are:  [-1, 0, 1, 2, 3.22]
b1: -7.65 (0.48)
b2: 16.34 (29.98)
b3: 0.34 (0.04)
b4: -15.42 (29.99)
gamma: -0.78 (2.24)



# c-d)

In [71]:
gg = np.linspace(2, 10, 400)

sses = []

for gamma in gg:
  X = x.copy()
  X[:, -1] = X[:, -1] / (1 + np.exp(gamma - X[:, -1]))

  sse = np.square(y - X @ np.linalg.inv(X.T @ X) @ X.T @ y).mean()
  sses.append(sse)

gamma = gg[np.argmin(sses)]
print("Optimal gamma is", round(gamma, 2))

Optimal gamma is 6.87


Close to what we obtained in b).

In [72]:
X = x.copy()
X[:, -1] = X[:, -1] / (1 + np.exp(gamma - X[:, -1]))

beta = np.linalg.inv(X.T @ X) @ X.T @ y
b2 = np.concatenate([beta, [gamma]])

se = np.diag(get_Vbeta(b2) / n) ** 0.5

for name, b, s in zip(names, b2, se):
  print(name, ": ", round(b, 2), " (", round(s, 2), ")" , sep='')

b1: -5.32 (0.49)
b2: 0.44 (0.1)
b3: 0.37 (0.04)
b4: 0.22 (0.05)
gamma: 6.87 (0.36)


# e)

In [73]:
def grad_logQ(x, b):
  eg = np.exp(b[-1])
  ex = np.exp(x)
  return b[1] + b[-2] * ex * (np.exp(b[-1]) * (x + 1) + ex) / (eg + ex) ** 2

In [74]:
#using values obtained in c)

print("Average Marginal Influence: ",
      round(grad_logQ(x[:, -1], b2).mean(), 2), end='')

print()

print("Marginal Influence at Average: ",
      round(grad_logQ(x.mean(axis=0)[-1], b2), 2), end='')

Average Marginal Influence:  0.8
Marginal Influence at Average:  0.89

Marginal influences differ by about 10%.

# f)

See pdf file.